## In this notebook:

#### We concatenate parcellated PET images into region x receptor matrix of densities.
Adapted from Hansen Receptors (https://github.com/netneurolab/hansen_receptors)
Paper: https://www.nature.com/articles/s41593-022-01186-3

### Import Packages

In [1]:
import numpy as np
import pandas as pd
from netneurotools import datasets, plotting
from matplotlib.colors import ListedColormap
from scipy.stats import zscore
from nilearn.datasets import fetch_atlas_schaefer_2018
from netneurotools.datasets import fetch_cammoun2012

In [2]:
import surfer

### Set Variables

In [3]:
path = '/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/'
datapath = '/Users/pecsok/projects/Neuromaps/hansen_receptors/'

atlas = 'cammoun'

if atlas == 'schaefer':
    schaefer = fetch_atlas_schaefer_2018(n_rois=1000, yeo_networks=17) # This is sklearn.utils._bunch.Bunch. Other altases in this format? *Ask Golia
    nnodes = len(schaefer['labels'])
elif atlas == 'cammoun':
    scale = 'cammoun2012_scale500' #'scale1000_17' #'Harvard_Oxford_sub' 
    cammoun = fetch_cammoun2012()
    nnodes=1015
elif atlas == 'Harvard_Oxford_sub':
    nnodes = 21


In [4]:
print(path+'data/parcellated/PET_parcellated/'+scale+'/mGluR5_abp_hc73_smart.csv')

/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/parcellated/PET_parcellated/cammoun2012_scale500/mGluR5_abp_hc73_smart.csv


### Choose what to analyse

In [5]:
receptors_csv = [path+'data/parcellated/PET_parcellated/'+scale+'/NMDA_ge179_hc29_galovic.csv',
                 path+'data/parcellated/PET_parcellated/'+scale+'/mGluR5_abp_hc22_rosaneto.csv',
                 path+'data/parcellated/PET_parcellated/'+scale+'/mGluR5_abp_hc28_dubois.csv',
                 path+'data/parcellated/PET_parcellated/'+scale+'/mGluR5_abp_hc73_smart.csv',
                # path+'data/parcellated/PET_parcellated/'+scale+'/GABAa-bz_flumazenil_hc16_norgaard.csv',
                 path+'data/parcellated/PET_parcellated/'+scale+'/GABAa_flumazenil_hc6_dukart.csv'

]

#                 path+'data/parcellated/PET_parcellated/'+scale+'/D2_flb457_hc37_smith.csv',
#                 path+'data/parcellated/PET_parcellated/'+scale+'/D2_flb457_hc55_sandiego.csv'


#path+'data/parcellated/PET_parcellated/'+scale+'D1_SCH23390_hc13_kaller.nii',
#path+'data/parcellated/PET_parcellated/'+scale+'D2_fallypride_hc49_jaworska.nii',
#path+'data/parcellated/PET_parcellated/'+scale+'D2_raclopride_hc7_alakurtti.nii',
# path+'data/parcellated/PET_parcellated/'+scale+'DAT_fpcit_hc174_dukart_spect.nii',
#path+'data/parcellated/PET_parcellated/'+scale+'DAT_fepe2i_hc6_sasaki.nii.gz'

### Make Receptor Matrices

In [6]:
print(scale) #pecsok_pfns/neuromaps/data/parcellated/PET_parcellated/"+scale+"

cammoun2012_scale500


In [12]:
# combine all the receptors (including repeats)
r = np.zeros([nnodes, len(receptors_csv)])
print(r.shape)
for i in range(len(receptors_csv)):
    r[:, i] = np.genfromtxt(receptors_csv[i], delimiter=',') 
    # Now we have the receptor data by parcel (row) for all 6 receptor maps (col)

#print(r[:,3])
print(r.shape)

receptor_names = np.array(["NMDA","mGluR5","GABAa"]) #,"D2"
np.save(path+'data/receptor_names_pet.npy', receptor_names)

# make final region x receptor matrix
receptor_data = np.zeros([nnodes, len(receptor_names)])

# NMDA Data
receptor_data[:, 0] = zscore(r[:, 0])
# weighted average of mGluR5 ABP688
receptor_data[:, 1] = (zscore(r[:, 1])*22 + zscore(r[:, 2])*28 + zscore(r[:, 3])*73) / (22+28+73)
# weighted average of GABA 
receptor_data[:, 2] = zscore(r[:, 4])


# weighted average of D2
#receptor_data[:, 3] = (zscore(r[:, 6])*37 + zscore(r[:, 7])*55) / (37+55)
# Weighted average of both GABAs:
# receptor_data[:, 2] = (zscore(r[:, 4])*16 + zscore(r[:, 5])*6) / (16+6)

# Make df and add parcel labels
receptor_df = pd.DataFrame(receptor_data, columns=receptor_names)
#schaefer = fetch_atlas_schaefer_2018(n_rois=1000, yeo_networks=17)
#labels = schaefer.labels
#labels = [label.decode('utf-8') for label in labels]

cam = fetch_cammoun2012()
info = pd.read_csv(cammoun['info'], sep=',')
info = info[info['scale']== 'scale500']
hemisphere = info['hemisphere'].values
labels = info['label'].values + hemisphere
print(labels)


receptor_df.index = labels
receptor_df.index.name = 'Parcel'


# Save df
receptor_df.to_csv(path+'results/receptor_data_'+scale+'.csv', index=True, header=True)

#print(receptor_df)

(1015, 5)
(1015, 5)
['lateralorbitofrontal_9R' 'lateralorbitofrontal_11R'
 'lateralorbitofrontal_5R' ... 'hippocampusL' 'amygdalaL' 'brainstemL']


In [13]:
"""
plot receptor data
"""

# colourmaps
cmap = np.genfromtxt(datapath+'data/colourmap.csv', delimiter=',')
cmap_div = ListedColormap(cmap)

# plot each receptor map
if scale == 'scale1000_17':
    annot = datasets.fetch_schaefer2018('fsaverage')['1000Parcels17Networks']
    type(annot)
    print(annot)
    
    for k in range(len(receptor_names)):
        brain = plotting.plot_fsaverage(data=receptor_data[:, k],
                                        lhannot=annot.lh,
                                        rhannot=annot.rh,
                                        colormap='plasma',
                                        views=['lat', 'med'],
                                        data_kws={'representation': "wireframe"})
        brain.save_image(path+'figures/scale1000_17/surface_receptor_'+receptor_names[k]+'.png')


if scale == 'scale100':
    annot = datasets.fetch_schaefer2018('fsaverage')['100Parcels7Networks']
    for k in range(len(receptor_names)):
        brain = plotting.plot_fsaverage(data=receptor_data[:, k],
                                        lhannot=annot.lh,
                                        rhannot=annot.rh,
                                        colormap='plasma',
                                        views=['lat', 'med'],
                                        data_kws={'representation': "wireframe"})
        brain.save_image(path+'figures/' + scale + '/surface_receptor_'+receptor_names[k]+'.png')

## NEWWWWWW
if scale == 'cammoun2012_scale500':
    annot = datasets.fetch_schaefer2018('fsaverage')['100Parcels7Networks']
    for k in range(len(receptor_names)):
        brain = plotting.plot_fsaverage(data=receptor_data[:, k],
                                        lhannot=annot.lh,
                                        rhannot=annot.rh,
                                        colormap='plasma',
                                        views=['lat', 'med'],
                                        data_kws={'representation': "wireframe"})
        brain.save_image(path+'figures/' + scale + '/surface_receptor_'+receptor_names[k]+'.png')




#receptor_data[:, 2:9] = r[:, 3:10]
print(receptor_data)
print(len(receptor_data))

print(r)

qt.qpa.window: <QNSWindow: 0x152050550; contentView=<QNSView: 0x31977b800; QCocoaWindow(0x600003b8c840, window=QWidgetWindow(0x6000028bff60, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.
qt.qpa.window: <QNSWindow: 0x16c45a760; contentView=<QNSView: 0x16c45a360; QCocoaWindow(0x600003b870d0, window=QWidgetWindow(0x6000029c2ee0, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.
qt.qpa.window: <QNSWindow: 0x373549d20; contentView=<QNSView: 0x373549920; QCocoaWindow(0x600003bab230, window=QWidgetWindow(0x6000029e5c20, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These wil

FileNotFoundError: [Errno 2] No such file or directory: '/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/figures/cammoun2012_scale500/surface_receptor_NMDA.png'